# Аналитика ДТП по предоставленным данным

Заказчиком (Оунером задачи) выступает проект «Карта ДТП» https://dtp-stat.ru/ — некоммерческий проект, посвященный проблеме дорожно-транспортных происшествий в России. Это платформа сбора данных о ДТП, бесплатный и открытый сервис аналитики ДТП.

**Цель проекта** - провести глубокий анализ данных, сформулировать и проверить гипотезы, опираясь на доступные признаки.

**Задачи**
* Провести исследовательский анализ данных о ДТП за 2025 год.
* Сформулировать и проверить не менее трех гипотез, основываясь на имеющихся признаках.
* Провести анализ ДТП по всем датасетам, представленным на сайте.
* Построить дашборд используя любой удобный инструмент, с учетом того, что дашборд может быть опубликован

**Декомпозиция работы:**
* Загрузка датасета и знакомство с данными
* Предобработка данных
* Исследовательский анализ данных
* Проверка гипотез
* Общие выводы
* Подготовка дашборда

## 1. Загрузка датасета и знакомство с данными

Загрузим необходимые библиотеки:

In [1]:
import os
import pandas as pd
import numpy as np
import gc

# Настройка отображения Jupiter
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)  # Отображать все строки
pd.set_option('display.max_columns', None)  # Отображать все столбцы
pd.set_option('display.width', None)  # Автоматическая ширина
pd.set_option('display.max_colwidth', None)  # Отображать полную ширину столбцов

**Для скачивания датасетов с сайта - необходимо использовать скрипт из ноутбука "DataLoader.ipynb"**

Загрузим данные:

In [2]:
all_records = []

download_dir = 'geojson_files'

# перебираем все .geojson в папке
for filename in os.listdir(download_dir):
    if filename.endswith(".geojson"):
        path = os.path.join(download_dir, filename)
        try:
            with open(path, encoding="utf-8") as f:
                data = json.load(f)
            accidents = data["features"]

            records = []
            for feature in accidents:
                props = feature["properties"].copy()
                props["geometry"] = feature["geometry"]
                records.append(props)

            all_records.extend(records)
            print(f"Загружено {len(records)} строк из {filename}")

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

# объединяем в один датафрейм
data = pd.DataFrame(all_records)

print(f"Итого строк во всех файлах: {len(data)}")

# Удалим переменную all_records:
del all_records

Загружено 10307 строк из novgorodskaia-oblast.geojson
Загружено 27892 строк из tiumenskaia-oblast.geojson
Загружено 5855 строк из sevastopol.geojson
Загружено 41694 строк из cheliabinskaia-oblast.geojson
Загружено 7015 строк из respublika-kareliia.geojson
Загружено 25787 строк из volgogradskaia-oblast.geojson
Загружено 20977 строк из vladimirskaia-oblast.geojson
Загружено 2323 строк из magadanskaia-oblast.geojson
Загружено 10181 строк из kurganskaia-oblast.geojson
Загружено 29859 строк из sverdlovskaia-oblast.geojson
Загружено 16533 строк из respublika-dagestan.geojson
Загружено 5274 строк из respublika-tyva.geojson
Загружено 2589 строк из chechenskaia-respublika.geojson
Загружено 12534 строк из arkhangelskaia-oblast.geojson
Загружено 5254 строк из respublika-kalmykiia.geojson
Загружено 13438 строк из tambovskaia-oblast.geojson
Загружено 28453 строк из omskaia-oblast.geojson
Загружено 6748 строк из kabardino-balkarskaia-respublika.geojson
Загружено 4274 строк из iamalo-nenetskii-avtono

Откроем датасет и посмотрим на данные:

In [3]:
data.head()

,id,tags,light,point,nearby,region,scheme,address,weather,category,datetime,severity,vehicles,dead_count,participants,injured_count,parent_region,road_conditions,participants_count,participant_categories,geometry
0,1504730,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 58.332904, 'long': 30.415138}",[],Шимский район,200,"Медведь-Батецкий, 7 км",[Ясно],Столкновение,2025-05-01 12:55:00,С погибшими,"[{'year': 2012, 'brand': 'KIA', 'color': 'Серый', 'model': 'Venga', 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'role': 'Водитель', 'gender': 'Женский', 'violations': ['Выезд на полосу встречного движения'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 4}, {'role': 'Пассажир', 'gender': 'Женский', 'violations': [], 'health_status': 'Не пострадал', 'years_of_driving_experience': None}]}, {'year': 2013, 'brand': 'Прочие марки мотоциклов', 'color': 'Красный', 'model': 'Прочие марки мотоциклов', 'category': 'Мопеды с двигателем внутреннего сгорания менее 50 см. куб.', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Управление ТС лицом, не имеющим права на управление ТС'], 'health_status': 'Скончался на месте ДТП до приезда скорой медицинской помощи', 'years_of_driving_experience': None}]}]",1,[],0,Новгородская область,[Сухое],3,"[Все участники, Мотоциклисты]","{'type': 'Point', 'coordinates': [30.415138, 58.332904]}"
1,156565,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': None, 'long': None}",[],Шимский район,600,"Батецкий-Медведь, 10 км",[Ясно],Опрокидывание,2015-06-11 16:10:00,Легкий,"[{'year': 2011, 'brand': 'KIA', 'color': 'Синий', 'model': 'Rio', 'category': 'D-класс (средний) до 4,6 м', 'participants': [{'role': 'Водитель', 'gender': 'Женский', 'violations': ['Другие нарушения ПДД водителем'], 'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)', 'years_of_driving_experience': 10}]}]",0,[],1,Новгородская область,[Сухое],1,[Все участники],"{'type': 'Point', 'coordinates': [None, None]}"
2,156274,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.295429, 'long': 30.494614}","[Жилые дома индивидуальной застройки, Нерегулируемый пешеходный переход, Остановка общественного транспорта, Нерегулируемый перекрёсток]",Шимский район,840,"д Старый Медведь, Медведь-Батецкий, 1 км",[Ясно],Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,"[{'year': 2009, 'brand': 'ВАЗ', 'color': 'Черный', 'model': 'Priora', 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Оставление места ДТП', 'Несоответствие скорости конкретным условиям движения'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 13}]}]",0,"[{'role': 'Пешеход', 'gender': 'Мужской', 'violations': [], 'health_status': 'Раненый, находящийся (находившийся) на стационарном лечении'}]",1,Новгородская область,[Сухое],2,"[Все участники, Пешеходы]","{'type': 'Point', 'coordinates': [30.494614, 58.295429]}"
3,156238,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.210136, 'long': 30.721529}","[Административные здания, Многоквартирные жилые дома, Нерегулируемый пешеходный переход]",Шимский район,740,"рп Шимск, ул Новгородская, 6",[Пасмурно],Наезд на пешехода,2022-12-06 17:05:00,Легкий,"[{'year': 2009, 'brand': 'CHEVROLET', 'color': 'Синий', 'model': 'Cruze', 'category': 'D-класс (средний) до 4,6 м', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Непредоставление преимущества в движении пешеходу', 'Другие нарушения ПДД водителем'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 25}]}]",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'violations': [], 'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневно

Посмотрим общую информацию о датасете:

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440070 entries, 0 to 1440069
Data columns (total 21 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   id                      1440070 non-null  int64 
 1   tags                    1440070 non-null  object
 2   light                   1440070 non-null  object
 3   point                   1440070 non-null  object
 4   nearby                  1440070 non-null  object
 5   region                  1440070 non-null  object
 6   scheme                  1361453 non-null  object
 7   address                 1364914 non-null  object
 8   weather                 1440070 non-null  object
 9   category                1440070 non-null  object
 10  datetime                1440070 non-null  object
 11  severity                1440070 non-null  object
 12  vehicles                1440070 non-null  object
 13  dead_count              1440070 non-null  int64 
 14  participants      

In [4]:
print(f'Исходный датасет содержит {data.shape[0]} записей и {data.shape[1]} признак(а/ов):')

Исходный датасет содержит 1440070 записей и 21 признак(а/ов):


* `id`: идентификатор
* `light`: время суток
* `point`: координаты (гео)
* `nearby`: комментарий по месту происшествия (где произошло)
* `region`: город/район
* `address`: адрес
* `weather`: погода
* `category`: тип ДТП
* `datetime`: дата и время происшествия
* `severity`: тяжесть ДТП/вред здоровью
* `vehicles`: участники – транспортные средства:
    * `year`: год производства транспортного средства
    * `brand`: марка транспортного средства
    * `color`: цвет транспортного средства
    * `model`: модель транспортного средства
    * `category`: категория транспортного средства
    * `role`: роль участника
    * `gender`: пол участника
    * `violations`: нарушения правил участником
    * `health_status`: состояние здоровья участника
    * `years_of_driving_experience`: стаж вождения участника (только у водителей)
* `dead_count`: кол-во погибших в ДТП
* `participants`: участники без транспортных средств (описание, как у участников внутри транспортных средств)
* `injured_count`: кол-во раненых в ДТП
* `parent_region`: родительский регион
* `road_conditions`: состояние дорожного покрытия
* `participants_count`: кол-во участников ДТП
* `participant_categories`: категории участников

Так как датасет имеет сложную структуру данных, где внутри одной записи (как в столбцах `vehicles` и `participants`) содержится информация о разных объектах (транспортное средство и участники ДТП) со своими отдельными свойствами, разобъем исходный датасет на две части - в первой части будет общая информация о ДТП, а во второй информация о транспортных средствах и участниках ДТП:

In [4]:
general_df = data.drop(columns=['vehicles','participants'])

detailed_df = data[['id', 'vehicles', 'participants']]

## 2. Предобработка данных

Задачи данного раздела:

* Привести типы данных в соответствие
* Проверить пропуски в датасете и обработать их
* Проверить наличие явных и неявных дубликатов и обработать их
* По возможности дообогатить данные на основе имеющихся

### 2.1. Предобработка датасета с общей информацией о ДТП

Посмотрим первые пять записей общего датасета:

In [7]:
general_df.head()

,id,tags,light,point,nearby,region,scheme,address,weather,category,datetime,severity,dead_count,injured_count,parent_region,road_conditions,participants_count,participant_categories,geometry
0,1504730,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 58.332904, 'long': 30.415138}",[],Шимский район,200,"Медведь-Батецкий, 7 км",[Ясно],Столкновение,2025-05-01 12:55:00,С погибшими,1,0,Новгородская область,[Сухое],3,"[Все участники, Мотоциклисты]","{'type': 'Point', 'coordinates': [30.415138, 58.332904]}"
1,156565,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': None, 'long': None}",[],Шимский район,600,"Батецкий-Медведь, 10 км",[Ясно],Опрокидывание,2015-06-11 16:10:00,Легкий,0,1,Новгородская область,[Сухое],1,[Все участники],"{'type': 'Point', 'coordinates': [None, None]}"
2,156274,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.295429, 'long': 30.494614}","[Жилые дома индивидуальной застройки, Нерегулируемый пешеходный переход, Остановка общественного транспорта, Нерегулируемый перекрёсток]",Шимский район,840,"д Старый Медведь, Медведь-Батецкий, 1 км",[Ясно],Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,0,1,Новгородская область,[Сухое],2,"[Все участники, Пешеходы]","{'type': 'Point', 'coordinates': [30.494614, 58.295429]}"
3,156238,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.210136, 'long': 30.721529}","[Административные здания, Многоквартирные жилые дома, Нерегулируемый пешеходный переход]",Шимский район,740,"рп Шимск, ул Новгородская, 6",[Пасмурно],Наезд на пешехода,2022-12-06 17:05:00,Легкий,0,1,Новгородская область,[Сухое],2,"[Все участники, Пешеходы]","{'type': 'Point', 'coordinates': [30.721529, 58.210136]}"
4,156240,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 58.067255, 'long': 30.765452}",[],Шимский район,610,"Шимск-Волот, 14 км",[Снегопад],Съезд с дороги,2021-11-27 15:00:00,Легкий,0,1,Новгородская область,"[Недостатки зимнего содержания, Заснеженное]",3,[Все участники],"{'type': 'Point', 'coordinates': [30.765452, 58.067255]}"


#### 2.1.1 Подготовка полей к дальнейшему анализу

В полях `tags`, `nearby`, `weather`, `road_conditions`, `participant_categories` содержатся списки, преобразуем данные поля к строковому типу.

Определим перечень полей для преобразования:

In [9]:
list_features = ['tags', 'nearby', 'weather', 'road_conditions', 'participant_categories']

Приведем поля к строковому типу:

In [10]:
for column in list_features:
    general_df[column] = general_df[column].apply(lambda x: x[0] if len(x) == 1 else ', '.join(x) if isinstance(x, list) else np.nan)

Проверим результат:

In [11]:
general_df.head()

,id,tags,light,point,nearby,region,scheme,address,weather,category,datetime,severity,dead_count,injured_count,parent_region,road_conditions,participants_count,participant_categories,geometry
0,1504730,Дорожно-транспортные происшествия,Светлое время суток,"{'lat': 58.332904, 'long': 30.415138}",,Шимский район,200,"Медведь-Батецкий, 7 км",Ясно,Столкновение,2025-05-01 12:55:00,С погибшими,1,0,Новгородская область,Сухое,3,"Все участники, Мотоциклисты","{'type': 'Point', 'coordinates': [30.415138, 58.332904]}"
1,156565,Дорожно-транспортные происшествия,Светлое время суток,"{'lat': None, 'long': None}",,Шимский район,600,"Батецкий-Медведь, 10 км",Ясно,Опрокидывание,2015-06-11 16:10:00,Легкий,0,1,Новгородская область,Сухое,1,Все участники,"{'type': 'Point', 'coordinates': [None, None]}"
2,156274,Дорожно-транспортные происшествия,"В темное время суток, освещение включено","{'lat': 58.295429, 'long': 30.494614}","Жилые дома индивидуальной застройки, Нерегулируемый пешеходный переход, Остановка общественного транспорта, Нерегулируемый перекрёсток",Шимский район,840,"д Старый Медведь, Медведь-Батецкий, 1 км",Ясно,Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,0,1,Новгородская область,Сухое,2,"Все участники, Пешеходы","{'type': 'Point', 'coordinates': [30.494614, 58.295429]}"
3,156238,Дорожно-транспортные происшествия,"В темное время суток, освещение включено","{'lat': 58.210136, 'long': 30.721529}","Административные здания, Многоквартирные жилые дома, Нерегулируемый пешеходный переход",Шимский район,740,"рп Шимск, ул Новгородская, 6",Пасмурно,Наезд на пешехода,2022-12-06 17:05:00,Легкий,0,1,Новгородская область,Сухое,2,"Все участники, Пешеходы","{'type': 'Point', 'coordinates': [30.721529, 58.210136]}"
4,156240,Дорожно-транспортные происшествия,Светлое время суток,"{'lat': 58.067255, 'long': 30.765452}",,Шимский район,610,"Шимск-Волот, 14 км",Снегопад,Съезд с дороги,2021-11-27 15:00:00,Легкий,0,1,Новгородская область,"Недостатки зимнего содержания, Заснеженное",3,Все участники,"{'type': 'Point', 'coordinates': [30.765452, 58.067255]}"


Удалим переменную `list_features`:

In [12]:
del list_features

Проверим уникальные значения по полю `tags`:

In [14]:
general_df['tags'].unique()

array(['Дорожно-транспортные происшествия'], dtype=object)

Данное поле не несет смысловой нагрузки, удалим его:

In [15]:
general_df = general_df.drop(columns='tags')

Распарсим поле `point`:

In [16]:
# Преобразование словарей в отдельные столбцы
location_df = general_df['point'].apply(pd.Series)

# Объединяем с оригинальным DataFrame
general_df = pd.concat([general_df.drop(columns=['point']), location_df], axis=1)

# Удалим переменную location_df
del location_df

Удалим дублирующий столбец с координатами:

In [17]:
general_df = general_df.drop(columns='geometry')

Фактический точный адрес ДТП для целей анализа малоприменим, в датасете есть данные о координатах происшествия, поэтому удалим колонку с адресом:

In [20]:
general_df = general_df.drop(columns='address')

#### 2.1.2. Обработка пропусков данных

Проверим наличие пропусков в датасете:

In [23]:
data_size = general_df.shape[0]

print(20*'==')
print('Пропуски, кол-во')
print(general_df.isnull().sum())
print(20*'==')
print('Пропуски, %')
print(general_df.isna().sum() / data_size * 100)
print(20*'==')

Пропуски, кол-во
id                            0
light                         0
nearby                        0
region                        0
scheme                    78617
weather                       0
category                      0
datetime                      0
severity                      0
dead_count                    0
injured_count                 0
parent_region                 0
road_conditions               0
participants_count            0
participant_categories        0
lat                        9792
long                       9792
dtype: int64
Пропуски, %
id                        0.000000
light                     0.000000
nearby                    0.000000
region                    0.000000
scheme                    5.459249
weather                   0.000000
category                  0.000000
datetime                  0.000000
severity                  0.000000
dead_count                0.000000
injured_count             0.000000
parent_region             0.0

Проверим какие данные содержатся в поле `scheme`:

In [24]:
general_df['scheme'].unique()

array(['200', '600', '840', '740', '610', '900', None, '070', '910',
       '500', '210', '950', '940', '440', '140', '830', '430', '820',
       '850', '010', '120', '130', '930', '420', '810', '220', '300',
       '030', '880', '060', '410', '050', '090', '920', '710', '860',
       '400', '800', '700', '020', '870', '230', '770', '760', '960',
       '720', '730', '320', '980', '750', '330', '110', '040', '620',
       '970', '630', '780', '100', '190', '340', '310'], dtype=object)

Здесь в основном представлены трехзначные числа в формате строки. На сайте ГИБДД отсутствует информация о кодировании схем ДТП. Заполним пропуски заглушкой "unknown":

In [25]:
general_df['scheme'] = general_df['scheme'].fillna('unknown')

Пропуски по полям координат `lat` и `long`. Так как в процентном соотношении таких пропусков менее 1% - удалим такие записи:

In [26]:
general_df = general_df.dropna(subset=['lat', 'long'])

#### 2.1.3. Обработка типов данных полей

Категориальный тип данных требует меньше памяти по сравнению со строковым типом. Поэтому приведем категориальные столбцы к соответствующему типу данных. Определим список категориальных полей:

In [28]:
general_cat_features = ['region', 'scheme', 'weather', 'category', 'severity', 'parent_region']

Преобразуем поля к категориальному типу данных pandas:

In [29]:
for column in general_cat_features:
    general_df[column] =  general_df[column].astype('category')

# Удалим переменную из памяти
del general_cat_features

Проверим максимальные и минимальные значения числовых полей:

In [33]:
# Отберем числовые поля
general_num_features = general_df.select_dtypes(include=['number']).columns.tolist()

# Посчитаем минимальное и максимальное значение
for feature in general_num_features:
    print(f'Признак: {feature} MIN:{general_df[feature].min()} MAX:{general_df[feature].max()}')

# Удалим переменную из памяти
del general_num_features

Признак: id MIN:1 MAX:1523161
Признак: dead_count MIN:0 MAX:20
Признак: injured_count MIN:0 MAX:106
Признак: participants_count MIN:1 MAX:168
Признак: lat MIN:0.0003 MAX:90.0
Признак: long MIN:-179.474244 MAX:179.640841


Для оптимизации используемой памяти приведем поля `dead_count`, `injured_count` к типу int8:

In [34]:
general_df['dead_count'] = general_df['dead_count'].astype('int8')

general_df['injured_count'] = general_df['injured_count'].astype('int8')

Приведем поле `datetime` к типу даты:

In [42]:
general_df['datetime'] = pd.to_datetime(general_df['datetime'])

Проверим полученный результат:

In [43]:
general_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1430278 entries, 0 to 1440069
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   id                      1430278 non-null  int64         
 1   light                   1430278 non-null  object        
 2   nearby                  1430278 non-null  object        
 3   region                  1430278 non-null  category      
 4   scheme                  1430278 non-null  category      
 5   weather                 1430278 non-null  category      
 6   category                1430278 non-null  category      
 7   datetime                1430278 non-null  datetime64[ns]
 8   severity                1430278 non-null  category      
 9   dead_count              1430278 non-null  int8          
 10  injured_count           1430278 non-null  int8          
 11  parent_region           1430278 non-null  category      
 12  road_conditions    

#### 2.1.4. Обработка дубликатов

Проверим наличие дубликатов:

In [37]:
general_df.duplicated().sum()

np.int64(0)

Явных дубликатов (полных совпадений строк не обнаружено).

Проверим дубликаты идентификаторов:

In [38]:
general_df['id'].duplicated().sum()

np.int64(0)

Дубликатов идентифиакторов не обнаружено, все записи имеют уникаьный идентификатор.

#### 2.1.5. Дообогащение данных

Добавим отдельные поля с днем, годом, месяцем и часом возникновения ДТП:

In [44]:
general_df['year'] = general_df['datetime'].dt.year
general_df['month'] = general_df['datetime'].dt.month
general_df['day'] = general_df['datetime'].dt.day
general_df['hour'] = general_df['datetime'].dt.hour

Проверим уникальные значения по полю `light`:

In [46]:
general_df['light'].unique()

array(['Светлое время суток', 'В темное время суток, освещение включено',
       'В темное время суток, освещение отсутствует', 'Сумерки',
       'Не установлено', 'В темное время суток, освещение не включено'],
      dtype=object)

Создадим следующие бинарные поля признаков освещения:
* Светлое время - `light_day`
* Темное время суток, освещение включено - `dark_light_on`
* В темное время суток, освещение отсутствует - `dark_no_light`
* В темное время суток, освещение не включено - `dark_light_off`
* Сумерки - `twilight`

In [49]:
general_df['light_day'] = general_df.apply(lambda x: 1 if x['light']=="Светлое время суток" else 0, axis=1)

general_df['dark_light_on'] = general_df.apply(lambda x: 1 if x['light']=="В темное время суток, освещение включено" else 0, axis=1)

general_df['dark_light_off'] = general_df.apply(lambda x: 1 if x['light']=="В темное время суток, освещение не включено" else 0, axis=1)

general_df['dark_no_light'] = general_df.apply(lambda x: 1 if x['light']=="В темное время суток, освещение отсутствует" else 0, axis=1)

Проверим уникальные значения по полю `nearby`:

In [61]:
unique_nearby = general_df['nearby'].str.split(', ').explode().unique()

unique_nearby_count = len(unique_nearby)

unique_nearby_count

unique_nearby

array(['', 'Жилые дома индивидуальной застройки',
       'Нерегулируемый пешеходный переход',
       'Остановка общественного транспорта', 'Нерегулируемый перекрёсток',
       'Административные здания', 'Многоквартирные жилые дома', 'Мост',
       'эстакада', 'путепровод',
       'Нерегулируемый перекрёсток неравнозначных улиц (дорог)',
       'Объект (здание', 'сооружение) религиозного культа',
       'Объект торговли', 'общественного питания на автодороге вне НП',
       'Выезд с прилегающей территории',
       'Крупный торговый объект (являющийся объектом массового тяготения пешеходов и (или) транспорта)',
       'Регулируемый перекрёсток', 'Регулируемый пешеходный переход',
       'Нерегулируемый перекрёсток равнозначных улиц (дорог)', 'Тротуар',
       'пешеходная дорожка', 'Иной объект', 'Подход к мосту', 'эстакаде',
       'путепроводу', 'Внутридворовая территория', 'АЗС',
       'Пункт весового контроля', 'Одиночный торговый объект',
       'являющийся местом притяжения транспо

Унифиуируем написание категорий:

In [58]:
general_df['nearby'] = general_df['nearby'].str.replace("перекресток", "перекрёсток")

Выделим следующие основные категории локаций ДТП:

* Пешеходный переходы:
    * Нерегулируемый пешеходный переход
    * Регулируемый пешеходный переход

* Перекрестки:
    * Нерегулируемый перекрёсток
    * Нерегулируемый перекрёсток неравнозначных улиц (дорог)
    * Регулируемый перекрёсток

* ЖД переезды:
    * Нерегулируемый ж/д переезд
    * Регулируемый ж/д переезд без дежурного
    * Регулируемый ж/д переезд с дежурным

* Круговое движение:
    * Нерегулируемое пересечение с круговым движением

А также добавим признак регулируемости.

In [68]:
# Признак перекрестка
general_df['crossroads'] = general_df.apply(lambda x: 1 if 'перекрёсток' in x['nearby'].lower() else 0, axis=1)

# Признак пешеходного перехода
general_df['pedestrian_crossing'] = general_df.apply(lambda x: 1 if 'пешеходный переход' in x['nearby'].lower() else 0, axis=1)

# Признак ЖД переезда
general_df['railway_crossing'] = general_df.apply(lambda x: 1 if 'ж/д переезд' in x['nearby'].lower() else 0, axis=1)

# Признак кругового движения
general_df['circular_motion'] = general_df.apply(lambda x: 1 if 'круг' in x['nearby'].lower() else 0, axis=1)

# Признак регулируемости
general_df['regulated'] = general_df.apply(lambda x: 1 if 'регулируемый' in x['nearby'].lower() and 'нерегулируемый' not in x['nearby'].lower() else 0, axis=1)

Проверим уникальные значения по полю `weather`:

In [71]:
general_df['weather'].unique().tolist()

['Ясно',
 'Пасмурно',
 'Снегопад',
 'Метель',
 'Дождь',
 'Туман',
 'Пасмурно, Дождь',
 'Ясно, Температура выше +30С',
 'Пасмурно, Снегопад',
 'Дождь, Снегопад',
 'Ясно, Температура ниже -30С',
 'Пасмурно, Туман',
 'Ясно, Туман',
 'Дождь, Туман',
 'Ясно, Ураганный ветер',
 'Пасмурно, Метель',
 'Пасмурно, Температура ниже -30С',
 'Снегопад, Метель',
 'Метель, Температура ниже -30С',
 'Ясно, Метель',
 'Пасмурно, Ураганный ветер',
 'Ясно, Дождь',
 'Снегопад, Температура ниже -30С',
 'Метель, Ураганный ветер',
 'Дождь, Температура выше +30С',
 'Туман, Снегопад',
 'Дождь, Ураганный ветер',
 'Дождь, Метель',
 'Туман, Температура ниже -30С',
 'Снегопад, Ураганный ветер',
 'Пасмурно, Температура выше +30С',
 'Туман, Метель',
 'Дождь, Температура ниже -30С']

На основе данного поля создадим несколько категориальных бинарных признаков, выделяющих факты неблагоприятных погодных явлений:
* Ясная погода
* Пасмурная погода
* Снегопад
* Метель
* Дождь
* Туман
* Ураганный ветер

In [80]:
# Признак ясной погоды
general_df['is_clear'] = general_df.apply(lambda x: 1 if 'ясно' in x['weather'].lower() else 0, axis=1)

# Признак пасмурной погоды
general_df['is_cloudy'] = general_df.apply(lambda x: 1 if 'пасмурно' in x['weather'].lower() else 0, axis=1)

# Признак снегопада
general_df['is_snowfall'] = general_df.apply(lambda x: 1 if 'cнегопад' in x['weather'].lower() else 0, axis=1)

# Признак метели
general_df['is_snowstorm'] = general_df.apply(lambda x: 1 if 'метель' in x['weather'].lower() else 0, axis=1)

# Признак дождя
general_df['is_rain'] = general_df.apply(lambda x: 1 if 'дождь' in x['weather'].lower() else 0, axis=1)

# Признак тумана
general_df['is_fog'] = general_df.apply(lambda x: 1 if 'туман' in x['weather'].lower() else 0, axis=1)

# Признак ураганного ветра
general_df['is_hurricane'] = general_df.apply(lambda x: 1 if 'ураган' in x['weather'].lower() else 0, axis=1)

Проверим уникальные значения по полю `category`:

In [83]:
general_df['category'].unique().tolist()

['Столкновение',
 'Наезд на пешехода',
 'Съезд с дороги',
 'Опрокидывание',
 'Наезд на животное',
 'Наезд на стоящее ТС',
 'Наезд на препятствие',
 'Наезд на велосипедиста',
 'Падение пассажира',
 'Иной вид ДТП',
 'Отбрасывание предмета',
 'Наезд на внезапно возникшее препятствие',
 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее несение службы',
 'Падение груза',
 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее какую-либо другую деятельность',
 'Наезд на лицо, не являющееся участником дорожного движения, осуществляющее производство работ',
 'Возгорание вследствие технической неисправности движущегося или остановившегося ТС, участвующего в дорожном движении.',
 'Наезд на гужевой транспорт']

Из данного признака выделим дополнительные бинарные признаки:
* Наезд / столкновение с человеком
* Наезд / столкновение с велосипедистом
* Наезд / столкновение с животным
* Возгорание

In [84]:
# Признак наезда / столкновения с человеком
general_df['is_collision_with_human'] = general_df.apply(lambda x: 1 if 'пешеход' in x['category'].lower() or 
                                                         'наезд на лицо' in x['category'].lower()
                                                         else 0, axis=1)

# Признак наезда / столкновения с велосипедистом
general_df['is_collision_with_cyclist'] = general_df.apply(lambda x: 1 if 'велосипедист' in x['category'].lower() else 0, axis=1)

# Признак наезда / столкновения с животным
general_df['is_collision_with_animal'] = general_df.apply(lambda x: 1 if 'животн' in x['category'].lower() else 0, axis=1)

# Признак возгорания
general_df['is_fire'] = general_df.apply(lambda x: 1 if 'возгорание' in x['category'].lower() else 0, axis=1)

Проверим уникальные значения по полю `severity`:

In [92]:
general_df['severity'].unique().tolist()

['С погибшими', 'Тяжёлый', 'Легкий']

Проверим уникальные значения по полю `road_conditions`:

In [94]:
general_df['road_conditions'].unique().tolist()

['Сухое',
 'Недостатки зимнего содержания, Заснеженное',
 'Мокрое',
 'Заснеженное',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Мокрое',
 'Недостатки зимнего содержания, Гололедица',
 'Обработанное противогололедными материалами',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое, Дефекты покрытия',
 'Сухое, Неудовлетворительное состояние обочин',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое, Отсутствие дорожных знаков в необходимых местах, Неудовлетворительное состояние обочин, Отсутствие тротуаров (пешеходных дорожек)',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое, Неправильное применение, плохая видимость дорожных знаков, Неудовлетворительное состояние обочин, Отсутствие тротуаров (пешеходных дорожек)',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое',
 'Сухое, Неправильное применение, плохая видимость дорожных знаков',
 'О

Поле содержит большое количество комбинаций описания состояния дорожного покрытия. Посчитаем сколько уникальных характеристик содержится - разобъем описание на части между запятыми и посчитаем количество уникальных характеристик:

In [97]:
unique_combinations = general_df['road_conditions'].str.split(', ').explode().unique()
unique_count = len(unique_combinations)
print(f'Получилось {unique_count} уникальных характеристик состояния дорожного покрытия')

Получилось 49 уникальных характеристик состояния дорожного покрытия


Посмотрим какие характерисстики получились:

In [98]:
unique_combinations

array(['Сухое', 'Недостатки зимнего содержания', 'Заснеженное', 'Мокрое',
       'Отсутствие',
       'плохая различимость горизонтальной разметки проезжей части',
       'Гололедица', 'Обработанное противогололедными материалами',
       'Дефекты покрытия', 'Неудовлетворительное состояние обочин',
       'Отсутствие дорожных знаков в необходимых местах',
       'Отсутствие тротуаров (пешеходных дорожек)',
       'Неправильное применение', 'плохая видимость дорожных знаков',
       'Иные недостатки', 'Отсутствие освещения',
       'Отсутствие элементов обустройства остановочного пункта общественного пассажирского транспорта',
       'Со снежным накатом', 'Неисправное освещение',
       'Сужение проезжей части', 'наличие препятствий',
       'затрудняющих движение транспортных средств',
       'плохая различимость вертикальной разметки', 'Загрязненное',
       'Неровное покрытие', 'Плохая видимость световозвращателей',
       'размещенных на дорожных ограждениях',
       'Отсутствие дор

Из данных признаков можно выделить следующие ключевые характеристики для создания отдельных категориальных бинарных прзнаков состояния дорожного покрытия:
* Гололедица, заснеженное, снежный накат
* Отсутствие освещения, неисправное освещение, недостаточное освещение,
* Дефекты покрытия, неровное покрытие
* Неисправность светофора, плохая видимость светофора

In [100]:
# Признак проблем связанных со снегом и обледенением
general_df['ice_troubles'] = general_df.apply(lambda x: 1 if 'гололед' in x['road_conditions'].lower() or 
                                                         'снеж' in x['road_conditions'].lower()
                                                         else 0, 
                                              axis=1)

# Признак проблем связанных с освещением
general_df['light_troubles'] = general_df.apply(lambda x: 1 if 'освещен' in x['road_conditions'].lower()
                                                         else 0, 
                                                axis=1)

# Признак проблем связанных с исправностью дорожного покрытия
general_df['road_coating_troubles'] = general_df.apply(lambda x: 1 if 'покрыт' in x['road_conditions'].lower()
                                                         else 0, 
                                                axis=1)

# Признак проблем связанных со светофорами
general_df['traffic_lights_troubles'] = general_df.apply(lambda x: 1 if 'светофор' in x['road_conditions'].lower()
                                                         else 0, 
                                                axis=1)

Удалим переменные unique_combinations, unique_count:

In [ ]:
del unique_combinations
del unique_count

Проверим уникальные значения по полю `participant_categories`:

In [102]:
general_df['participant_categories'].unique()

array(['Все участники, Мотоциклисты', 'Все участники, Пешеходы',
       'Все участники', 'Все участники, Велосипедисты',
       'Все участники, Велосипедисты, Мотоциклисты',
       'Все участники, Пешеходы, Мотоциклисты',
       'Все участники, Общ. транспорт',
       'Все участники, Пешеходы, Общ. транспорт', 'Мотоциклисты', '',
       'Все участники, Пешеходы, Велосипедисты',
       'Все участники, Мотоциклисты, Общ. транспорт',
       'Все участники, Велосипедисты, Общ. транспорт',
       'Все участники, Пешеходы, Велосипедисты, Мотоциклисты', 'Пешеходы',
       'Все участники, Пешеходы, Мотоциклисты, Общ. транспорт',
       'Все участники, Пешеходы, Велосипедисты, Общ. транспорт'],
      dtype=object)

Из данного поля можно выделить следующие категориальные признаки:
* Мотоциклисты
* Пешеходы
* Велосипедисты
* Общ. транспорт

In [103]:
# Признак мотоциклиста
general_df['participant_biker'] = general_df.apply(lambda x: 1 if 'мотоцикл' in x['participant_categories'].lower()
                                                         else 0, 
                                                axis=1)

# Признак пешехода
general_df['participant_pedestrian'] = general_df.apply(lambda x: 1 if 'пешеход' in x['participant_categories'].lower()
                                                         else 0, 
                                                axis=1)

# Признак велосипедиста
general_df['participant_cyclist'] = general_df.apply(lambda x: 1 if 'велосипед' in x['participant_categories'].lower()
                                                         else 0, 
                                                axis=1)

# Признак общественного транспорта
general_df['participant_public_transport'] = general_df.apply(lambda x: 1 if 'общ.' in x['participant_categories'].lower()
                                                         else 0, 
                                                axis=1)

Проверим полученный результат:

In [104]:
general_df.head()

,id,light,nearby,region,scheme,weather,category,datetime,severity,dead_count,injured_count,parent_region,road_conditions,participants_count,participant_categories,lat,long,year,month,day,hour,light_day,dark_light_on,dark_light_off,dark_no_light,crossroads,pedestrian_crossing,railway_crossing,circular_motion,regulated,is_snowfall,is_snowstorm,is_rain,is_fog,is_hurricane,is_clear,is_cloudy,is_collision_with_human,is_collision_with_cyclist,is_collision_with_animal,is_fire,ice_troubles,light_troubles,road_coating_troubles,traffic_lights_troubles,participant_biker,participant_pedestrian,participant_cyclist,participant_public_transport
0,1504730,Светлое время суток,,Шимский район,200,Ясно,Столкновение,2025-05-01 12:55:00,С погибшими,1,0,Новгородская область,Сухое,3,"Все участники, Мотоциклисты",58.332904,30.415138,2025,5,1,12,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,156274,"В темное время суток, освещение включено","Жилые дома индивидуальной застройки, Нерегулируемый пешеходный переход, Остановка общественного транспорта, Нерегулируемый перекрёсток",Шимский район,840,Ясно,Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,0,1,Новгородская область,Сухое,2,"Все участники, Пешеходы",58.295429,30.494614,2022,10,7,20,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0
3,156238,"В темное время суток, освещение включено","Административные здания, Многоквартирные жилые дома, Нерегулируемый пешеходный переход",Шимский район,740,Пасмурно,Наезд на пешехода,2022-12-06 17:05:00,Легкий,0,1,Новгородская область,Сухое,2,"Все участники, Пешеходы",58.210136,30.721529,2022,12,6,17,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0
4,156240,Светлое время суток,,Шимский район,610,Снегопад,Съезд с дороги,2021-11-27 15:00:00,Легкий,0,1,Новгородская область,"Недостатки зимнего содержания, Заснеженное",3,Все участники,58.067255,30.765452,2021,11,27,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,156242,"В темное время суток, освещение отсутствует",,Шимский район,600,Пасмурно,Опрокидывание,2017-10-22 20:03:00,Легкий,0,1,Новгородская область,Мокрое,3,Все участники,58.193410,30.885960,2017,10,22,20,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


### 2.2. Предобработка датасета с детальной информацией об участниках ДТП

Посмотрим первые строки детального датасета с информацие о ДТП:

In [106]:
detailed_df.head()

,id,vehicles,participants
0,1504730,"[{'year': 2012, 'brand': 'KIA', 'color': 'Серый', 'model': 'Venga', 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'role': 'Водитель', 'gender': 'Женский', 'violations': ['Выезд на полосу встречного движения'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 4}, {'role': 'Пассажир', 'gender': 'Женский', 'violations': [], 'health_status': 'Не пострадал', 'years_of_driving_experience': None}]}, {'year': 2013, 'brand': 'Прочие марки мотоциклов', 'color': 'Красный', 'model': 'Прочие марки мотоциклов', 'category': 'Мопеды с двигателем внутреннего сгорания менее 50 см. куб.', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Управление ТС лицом, не имеющим права на управление ТС'], 'health_status': 'Скончался на месте ДТП до приезда скорой медицинской помощи', 'years_of_driving_experience': None}]}]",[]
1,156565,"[{'year': 2011, 'brand': 'KIA', 'color': 'Синий', 'model': 'Rio', 'category': 'D-класс (средний) до 4,6 м', 'participants': [{'role': 'Водитель', 'gender': 'Женский', 'violations': ['Другие нарушения ПДД водителем'], 'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)', 'years_of_driving_experience': 10}]}]",[]
2,156274,"[{'year': 2009, 'brand': 'ВАЗ', 'color': 'Черный', 'model': 'Priora', 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Оставление места ДТП', 'Несоответствие скорости конкретным условиям движения'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 13}]}]","[{'role': 'Пешеход', 'gender': 'Мужской', 'violations': [], 'health_status': 'Раненый, находящийся (находившийся) на стационарном лечении'}]"
3,156238,"[{'year': 2009, 'brand': 'CHEVROLET', 'color': 'Синий', 'model': 'Cruze', 'category': 'D-класс (средний) до 4,6 м', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Непредоставление преимущества в движении пешеходу', 'Другие нарушения ПДД водителем'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 25}]}]","[{'role': 'Пешеход', 'gender': 'Женский', 'violations': [], 'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара'}]"
4,156240,"[{'year': 2010, 'brand': 'ПАЗ', 'color': 'Белый', 'model': '3204', 'category': 'Одноэтажные длиной от 5 до 8 м', 'participants': [{'role': 'Пассажир', 'gender': 'Женский', 'violations': [], 'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара', 'years_of_driving_experience': None}, {'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Несоответствие скорости конкретным условиям движения', 'Эксплуатация ТС с техническими неисправностями, при которых запрещается их эксплуатация'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 24}, {'role': 'Пассажир', 'gender': 'Женский', 'violations': [], 'health_status': 'Получил травмы с оказанием разовой медицинской помощи, к категории раненый не относится', 'years_of_driving_experience': None}]}]",[]


Информация о транспортных средствах и участниках ДТП представлена в виде списков словарей разного размера. Распарсим поля `vehicles` и `participants` чтобы получилась плоская таблица.

#### 2.2.1. Парсинг полей со списком словарей

Напишем функциб для парсинга полей `vehicles` и `participants`:

In [111]:
def flatten_data(df):
    """Разворачивает данные из столбцов 'vehicles' и 'participants' в плоскую таблицу."""

    rows = []
    for _, row in df.iterrows():
        row_id = row['id']
        vehicles = row['vehicles']
        participants_external = row['participants']


        for vehicle in vehicles:
            vehicle_data = {f'vehicle_{k}': v for k, v in vehicle.items() if k != 'participants'}
            for participant in vehicle['participants']:
                participant_data = {f'participant_{k}': v for k, v in participant.items()}
                combined_data = {'id': row_id}
                combined_data.update(vehicle_data)
                combined_data.update(participant_data)
                rows.append(combined_data)

        for participant in participants_external:
            participant_data = {f'participant_{k}': v for k, v in participant.items()}
            combined_data = {'id': row_id}
            combined_data.update(participant_data)
            rows.append(combined_data)


    return pd.DataFrame(rows)

Распарсим данные:

In [118]:
participant_df = flatten_data(detailed_df)

Проверим результат:

In [119]:
participant_df.head()

,id,vehicle_year,vehicle_brand,vehicle_color,vehicle_model,vehicle_category,participant_role,participant_gender,participant_violations,participant_health_status,participant_years_of_driving_experience
0,1504730,2012.0,KIA,Серый,Venga,"В-класс (малый) до 3,9 м",Водитель,Женский,[Выезд на полосу встречного движения],Не пострадал,4.0
1,1504730,2012.0,KIA,Серый,Venga,"В-класс (малый) до 3,9 м",Пассажир,Женский,[],Не пострадал,NaN
2,1504730,2013.0,Прочие марки мотоциклов,Красный,Прочие марки мотоциклов,Мопеды с двигателем внутреннего сгорания менее 50 см. куб.,Водитель,Мужской,"[Управление ТС лицом, не имеющим права на управление ТС]",Скончался на месте ДТП до приезда скорой медицинской помощи,NaN
3,156565,2011.0,KIA,Синий,Rio,"D-класс (средний) до 4,6 м",Водитель,Женский,[Другие нарушения ПДД водителем],"Раненый, находящийся (находившийся) на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)",10.0
4,156274,2009.0,ВАЗ,Черный,Priora,"В-класс (малый) до 3,9 м",Водитель,Мужской,"[Оставление места ДТП, Несоответствие скорости конкретным условиям движения]",Не пострадал,13.0


Распарсим поле `participant_violations`:

In [120]:
participant_df['participant_violations'] = participant_df['participant_violations'].apply(lambda x: x[0] if len(x) == 1 else ', '.join(x) if isinstance(x, list) else np.nan)

#### 2.2.2. Обработка пропусков данных

Проверим наличие пропусков в датасете:

In [123]:
data_size = participant_df.shape[0]

print(20*'==')
print('Пропуски, кол-во')
print(participant_df.isnull().sum())
print(20*'==')
print('Пропуски, %')
print(participant_df.isna().sum() / data_size * 100)
print(20*'==')

Пропуски, кол-во
id                                               0
vehicle_year                                542669
vehicle_brand                               533150
vehicle_color                               502396
vehicle_model                               533176
vehicle_category                            445004
participant_role                                 0
participant_gender                           84490
participant_violations                           0
participant_health_status                     5901
participant_years_of_driving_experience    1581456
dtype: int64
Пропуски, %
id                                          0.000000
vehicle_year                               15.234499
vehicle_brand                              14.967269
vehicle_color                              14.103904
vehicle_model                              14.967999
vehicle_category                           12.492722
participant_role                            0.000000
participant_gender        

Видим большое количество пропущенных значений в характеристиках транспортных средств.

Пропуски по году выпуска заполним заглушкой "0":

In [134]:
participant_df['vehicle_year'] = participant_df['vehicle_year'].fillna(0)

Пропуски марки автомобиля заполним заглушкой:

In [124]:
participant_df['vehicle_brand'] = participant_df['vehicle_brand'].fillna('unknown')

Пропуски цвета ТС заполним заглушкой:

In [125]:
participant_df['vehicle_color'] = participant_df['vehicle_color'].fillna('unknown')

Пропуски модели ТС заполним заглушкой:

In [126]:
participant_df['vehicle_model'] = participant_df['vehicle_model'].fillna('unknown')

Пропуски категории ТС заполним заглушкой:

In [127]:
participant_df['vehicle_category'] = participant_df['vehicle_category'].fillna('unknown')

Пропуски в поле участника заполним заглушкой:

In [128]:
participant_df['participant_gender'] = participant_df['participant_gender'].fillna('unknown')

Пропуски в поле `participant_health_status` заполним заглушкой:

In [129]:
participant_df['participant_health_status'] = participant_df['participant_health_status'].fillna('unknown')

Пропуски по полю водительского стажа заполнять не будем.

#### 2.2.3. Обработка типов данных полей

Посмотрим общую информацию о датасете:

In [130]:
participant_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3562106 entries, 0 to 3562105
Data columns (total 11 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   id                                       int64  
 1   vehicle_year                             float64
 2   vehicle_brand                            object 
 3   vehicle_color                            object 
 4   vehicle_model                            object 
 5   vehicle_category                         object 
 6   participant_role                         object 
 7   participant_gender                       object 
 8   participant_violations                   object 
 9   participant_health_status                object 
 10  participant_years_of_driving_experience  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 298.9+ MB


Приведем поле `vehicle_year` к int16:

In [135]:
participant_df['vehicle_year'] = participant_df['vehicle_year'].astype('int16')

Приведем поле `participant_years_of_driving_experience` к float16:

In [132]:
participant_df['participant_years_of_driving_experience'] = participant_df['participant_years_of_driving_experience'].astype('float16')

Поля `vehicle_brand`, `vehicle_color`, `vehicle_model` , `vehicle_category`, `participant_role`, `participant_gender` приведем к категориальному типу:

In [137]:
participant_cat_features = ['vehicle_brand', 'vehicle_color', 'vehicle_model', 'vehicle_category', 'participant_role', 'participant_gender']

for column in participant_cat_features:
    participant_df[column] = participant_df[column].astype('category')

Проверим результат:

In [138]:
participant_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3562106 entries, 0 to 3562105
Data columns (total 11 columns):
 #   Column                                   Dtype   
---  ------                                   -----   
 0   id                                       int64   
 1   vehicle_year                             int16   
 2   vehicle_brand                            category
 3   vehicle_color                            category
 4   vehicle_model                            category
 5   vehicle_category                         category
 6   participant_role                         category
 7   participant_gender                       category
 8   participant_violations                   object  
 9   participant_health_status                object  
 10  participant_years_of_driving_experience  float16 
dtypes: category(6), float16(1), int16(1), int64(1), object(2)
memory usage: 122.4+ MB


В результате оптимизации типов данных использование памяти сократилось более чем в 2 раза.

#### 2.2.4. Обработка дубликатов

Проверим наличие дубликатов:

In [139]:
participant_df.duplicated().sum()

np.int64(130220)

Обнаружено 130 тыс. дубликатов. Посмотрим на эти данные:

In [140]:
participant_df[participant_df.duplicated(keep=False)].sort_values(by='id').head(20)

,id,vehicle_year,vehicle_brand,vehicle_color,vehicle_model,vehicle_category,participant_role,participant_gender,participant_violations,participant_health_status,participant_years_of_driving_experience
1781798,9,2020,HYUNDAI,Белый,Solaris,Прочие легковые автомобили,Пассажир,Женский,,"Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара",NaN
1781799,9,2020,HYUNDAI,Белый,Solaris,Прочие легковые автомобили,Пассажир,Женский,,"Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара",NaN
1781891,54,2015,MERCEDES,Белый,Прочие модели Mercedes,Прочая спецтехника,Пассажир,Мужской,,"Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара",NaN
1781890,54,2015,MERCEDES,Белый,Прочие модели Mercedes,Прочая спецтехника,Пассажир,Мужской,,"Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара",NaN
1781907,63,0,unknown,unknown,unknown,unknown,Пешеход,Женский,,"Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара",NaN
1781908,63,0,unknown,unknown,unknown,unknown,Пешеход,Женский,,"Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара",NaN
1788402,98,0,unknown,unknown,unknown,unknown,Пешеход,Женский,Переход через проезжую часть вне пешеходного перехода в зоне его видимости либо при наличии в непосредственной близости подземного (надземного) пешеходного перехода,"Раненый, находящийся (находившийся) на стационарном лечении",NaN
1788401,98,0,unknown,unknown,unknown,unknown,Пешеход,Женский,Переход через проезжую часть вне пешеходного перехода в зоне его видимости либо при наличии в непосредственной близости подземного (надземного) пешеходного перехода,"Раненый, находящийся (находившийся) на стационарном лечении",NaN
1788660,117,2005,ВАЗ,Иные цвета,"ВАЗ 2110, 21101, 21102, 21103, 21108","В-класс (малый) до 3,9 м",Пассажир,Женский,,"Раненый, находящийся (находившийся) на стационарном лечении",NaN
1788661,117,2005,ВАЗ,Иные цвета,"ВАЗ 2110, 21101, 21102, 21103, 21108","В-класс (малый) до 3,9 м",Пассажир,Женский,,"Раненый, находящийся (находившийся) на стационарном лечении",NaN


Видим, что действительно данные дублируются. Удалим дубликаты:

In [141]:
participant_df = participant_df.drop_duplicates()

Проверим уникальные значения по полю `vehicle_brand`:

In [145]:
participant_df['vehicle_brand'].unique().tolist()

['KIA',
 'Прочие марки мотоциклов',
 'ВАЗ',
 'unknown',
 'CHEVROLET',
 'ПАЗ',
 'RENAULT',
 'SKODA',
 'VOLVO',
 'PEUGEOT',
 'Datsun',
 'CITROEN',
 'НефАЗ',
 'VOLKSWAGEN',
 'BMW',
 'SSANGYONG',
 'TOYOTA',
 'ГАЗ',
 'LAND ROVER',
 'ISUZU',
 'MERCEDES',
 'Прочие марки ТС',
 'HYUNDAI',
 'MITSUBISHI',
 'ИЖ',
 'FIAT',
 'LEXUS',
 'AUDI',
 'DAEWOO',
 'OPEL',
 'ЗИЛ',
 'FORD',
 'КАМАЗ',
 'DODGE',
 'MAZDA',
 'DAIHATSU',
 'NISSAN',
 'YAMAHA',
 'HONDA',
 'SUZUKI',
 'CHERY',
 'KAWASAKI',
 'FORSAGE',
 'ВОСХОД',
 'JAC',
 'GEELY',
 'УАЗ',
 'ЗАЗ',
 'LIFAN',
 'RELIANT',
 'BAW',
 'HINDUSTAN',
 'DACIA',
 'MAN',
 'DAF',
 'МАЗ-МАН',
 'SCANIA',
 'CHRYSLER',
 'DONG FENG',
 'FAW',
 'SUBARU',
 'FREIGHTLINER',
 'IVECO',
 'CADILLAC',
 'SHACMAN',
 'ZX',
 'ROVER',
 'Прочие марки грузовых ТС',
 'ALFER',
 'ТАГАЗ (TAGAZ)',
 'FSO',
 'ABM',
 'GREAT WALL',
 'PORSCHE',
 'KENWORTH',
 'MOTOLEVO',
 'МАЗ',
 'Прочие марки седельных тягачей',
 'NEOPLAN',
 'Прочие марки легких коммерческих ТС',
 'REGAL RAPTOR',
 'BSA',
 'ALPINA',
 

Проверим уникальные значения по полю `vehicle_color`:

In [149]:
participant_df['vehicle_color'].unique().tolist()

['Серый',
 'Красный',
 'Синий',
 'Черный',
 'unknown',
 'Белый',
 'Зеленый',
 'Коричневый',
 'Желтый',
 'Иные цвета',
 'Оранжевый',
 'Фиолетовый',
 'Многоцветный']

Записи содержащие "не заполнено" заменим на "unknown":

In [148]:
participant_df['vehicle_color'] = participant_df['vehicle_color'].apply(lambda x: 'unknown' if 'Не заполнено' in x else x)

Проверим уникальные значения по полю `vehicle_model`:

In [152]:
participant_df['vehicle_model'].unique().tolist()

['Venga',
 'Прочие марки мотоциклов',
 'Rio',
 'Priora',
 'unknown',
 'Cruze',
 '3204',
 'Жигули  ВАЗ-2107 модификации',
 'Logan',
 'Largus (Ларгус)',
 'Fabia',
 '940',
 'Sandero',
 'ВАЗ 2111, 21110 (универсал), 21113, 21114',
 '308',
 'mi-DO',
 'Jumper',
 'Прочие пригородные',
 'ВАЗ 2112 и модификации',
 'Passat',
 'Прочие модели BMW',
 'Actyon',
 'Camry',
 'FH',
 'Прочие модели ГАЗ',
 'Ока  ВАЗ-1111 и модификации',
 'Granta (Гранта)',
 'Discovery',
 'Niva',
 'Жигули  ВАЗ-2108, 09 и модификации',
 'Прочие модели Isuzu',
 'Нива  ВАЗ-2121 и модификации',
 'Прочие модели Mercedes',
 'Жигули  ВАЗ-2106 модификации',
 'Kalina',
 'Прочие марки и модели ТС',
 'ix35',
 'L-series',
 'ИЖ 2126',
 'Albea',
 'Прочие модели Lexus',
 '3302, 33027 и модификации',
 '80',
 'Elantra (Lantra, Avante)',
 'Nexia',
 'Vectra',
 'Жигули  ВАЗ-2104 модификации',
 'Yaris',
 'Golf',
 'Прочие модели ЗИЛ',
 'Focus',
 '6520',
 'Caravan',
 'Corolla',
 'Sprinter',
 'Movano',
 'RAV 4',
 'А6',
 'Mazda 3',
 'Fusion',
 'Пр

Удалим лишние пробелы в начале или конце строк:

In [151]:
participant_df['vehicle_model'] = participant_df['vehicle_model'].apply(lambda x: x.strip())

Проверим уникальные значения по полю `vehicle_category`:

In [153]:
participant_df['vehicle_category'].unique().tolist()

['В-класс (малый) до 3,9 м',
 'Мопеды с двигателем внутреннего сгорания менее 50 см. куб.',
 'D-класс (средний) до 4,6 м',
 'unknown',
 'Одноэтажные длиной от 5 до 8 м',
 'Прочие легковые автомобили',
 'Фургоны',
 'Одноэтажные длиной от 8 до 12 м',
 'Седельные тягачи',
 'А-класс (особо малый) до 3,5 м',
 'Рефрижераторы',
 'Тракторы',
 'Бортовые грузовые автомобили',
 'С-класс (малый средний, компактный) до 4,3 м',
 'Мопеды с электродвигателем менее 4 кВт',
 'Велосипеды',
 'Квадроциклы',
 'Самосвалы',
 'Легковые автомобили (без типа)',
 'Мотоциклы',
 'Шасси',
 'Бортовые',
 'Прочие одноярусные',
 'Прочие',
 'Минивэны и универсалы повышенной вместимости',
 'Е-класс (высший средний, бизнес-класс) до 4,9 м',
 'Прочие грузовые автомобили',
 'S-класс (высший, представительский класс) более 4,9 м',
 'Экскаваторы',
 'Одноэтажные длиной не более 5 м',
 'Мопеды с двигателем внутреннего сгорания более 50 см. куб.',
 'Специализированная снегоуборочная техника',
 'Автомобили скорой медицинской помощ

Проверим уникальные значения по полю `participant_role`:

In [154]:
participant_df['participant_role'].unique().tolist()

['Водитель',
 'Пассажир',
 'Пешеход',
 'Велосипедист',
 'Иной участник',
 'Пешеход, перед ДТП находившийся в (на) ТС в качестве водителя или пешеход, перед ДТП находившийся в (на) ТС в качестве пассажира',
 'Сотрудник ДПС (ГИБДД), выполняющий служебные обязанности на проезжей части (обочине и т.д.)',
 'Сотрудник полиции (кроме ГИБДД), сотрудник (военнослужащий) Росгвардии, Минобороны, ФСБ, ФСО, МЧС и т.д., выполняющий служебные обязанности на проезжей части (обочине и т.д.)',
 'Работник дорожной организации, осуществляющий работы на проезжей части (обочине и т.д.)',
 'Всадник',
 'Лицо, осуществляющее торговлю (иную деятельность) на проезжей части (обочине и т.д.)',
 'Работник иных организаций (коммунальных служб, электросетей, водоканала и т.д.), осуществляющий работы на проезжей части (обочине и т.д.)',
 'Погонщик скота',
 'Лицо, осуществляющее умышленное перекрытие проезжей части']

Проверим уникальные значения по полю `participant_gender`:

In [155]:
participant_df['participant_gender'].unique().tolist()

['Женский', 'Мужской', 'unknown']

#### 2.2.5. Дообогащение данных

Проверим уникальные значения по полю `participant_violations`:

In [157]:
participant_df['participant_violations'].unique().tolist()

['Выезд на полосу встречного движения',
 '',
 'Управление ТС лицом, не имеющим права на управление ТС',
 'Другие нарушения ПДД водителем',
 'Оставление места ДТП, Несоответствие скорости конкретным условиям движения',
 'Непредоставление преимущества в движении пешеходу, Другие нарушения ПДД водителем',
 'Несоответствие скорости конкретным условиям движения, Эксплуатация ТС с техническими неисправностями, при которых запрещается их эксплуатация',
 'Нарушение правил расположения ТС на проезжей части, Несоблюдение требований ОСАГО, Управление ТС лицом, не имеющим права на управление ТС, Неповиновение или сопротивление находящемуся при исполнении служебных обязанностей сотруднику правоохранительных органов или военнослужащему при остановке или задержании ТС, Отказ водителя, не имеющего права управления ТС либо лишенного права управления ТС от прохождении медицинского освидетельствования на состояние опьянения',
 'Несоответствие скорости конкретным условиям движения',
 'Несоответствие скоро

Проверим уникальные комбинации:

In [158]:
unique_combinations = participant_df['participant_violations'].str.split(', ').explode().unique()
unique_count = len(unique_combinations)
print(f'Получилось {unique_count} уникальных нарушений ДТП')

Получилось 140 уникальных характеристик состояния дорожного покрытия


In [159]:
unique_combinations

array(['Выезд на полосу встречного движения', '', 'Управление ТС лицом',
       'не имеющим права на управление ТС',
       'Другие нарушения ПДД водителем', 'Оставление места ДТП',
       'Несоответствие скорости конкретным условиям движения',
       'Непредоставление преимущества в движении пешеходу',
       'Эксплуатация ТС с техническими неисправностями',
       'при которых запрещается их эксплуатация',
       'Нарушение правил расположения ТС на проезжей части',
       'Несоблюдение требований ОСАГО',
       'Неповиновение или сопротивление находящемуся при исполнении служебных обязанностей сотруднику правоохранительных органов или военнослужащему при остановке или задержании ТС',
       'Отказ водителя',
       'не имеющего права управления ТС либо лишенного права управления ТС от прохождении медицинского освидетельствования на состояние опьянения',
       'Нарушение водителем правил применения ремней безопасности (ставится в случае',
       'когда не пристегнут водитель)', 'Неп

Выделим следующие признаки нарушения правил ПДД:
* Употребление алкоголя, наркотических веществ, нахождение в состоянии алкогольного / наркотического опьянения
* Пользование мобильной связью во время управления автомобилем
* Превышение скорости движения
* Нарушение требований сигналов светофора

In [164]:
participant_df['is_intoxication'] = participant_df.apply(lambda x: 1 if 'алко' in x['participant_violations'].lower() or
                                                                       'нарко' in x['participant_violations'].lower() or
                                                                       'опьянен' in x['participant_violations'].lower()
                                                         else 0, 
                                                axis=1)

participant_df['is_use_mobcell'] = participant_df.apply(lambda x: 1 if 'мобильной связью' in x['participant_violations'].lower()
                                                         else 0, 
                                                axis=1)

participant_df['is_above_speed_limit'] = participant_df.apply(lambda x: 1 if 'превышение установленной скорости движения' in x['participant_violations'].lower()
                                                         else 0, 
                                                axis=1)

participant_df['is_traffic_light_violation'] = participant_df.apply(lambda x: 1 if 'нарушение требований сигналов светофора' in x['participant_violations'].lower()
                                                         else 0, 
                                                axis=1)

In [168]:
del unique_combinations
del unique_count

Проверим уникальные значения по полю `participant_health_status`:

In [167]:
participant_df['participant_health_status'].unique()

array(['Не пострадал',
       'Скончался на месте ДТП до приезда скорой медицинской помощи',
       'Раненый, находящийся (находившийся)  на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)',
       'Раненый, находящийся (находившийся) на стационарном лечении',
       'Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневного стационара',
       'Получил травмы с оказанием разовой медицинской помощи, к категории раненый не относится',
       'Получил телесные повреждения с показанием к лечению в медицинских организациях (кроме разовой медицинской помощи)',
       'Скончался на месте ДТП по прибытию скорой медицинской помощи, но до транспортировки в мед. организацию',
       'Скончался в течение 5 суток', 'Скончался в течение 20 суток',
       'Скончался в течение 3 суток', 'Скончался в течение 1 суток',
       'Скончался в течение 11 суток', 'unk

Из данного поля выделим следующие категории:
* скончался
* не пострадал
* получил травмы

In [169]:
participant_df['is_dead'] = participant_df.apply(lambda x: 1 if 'скончался' in x['participant_health_status'].lower()
                                                         else 0, 
                                                axis=1)

participant_df['is_not_injured'] = participant_df.apply(lambda x: 1 if 'не пострадал' in x['participant_health_status'].lower()
                                                         else 0, 
                                                axis=1)

participant_df['is_injured'] = participant_df.apply(lambda x: 1 if 'раненый' in x['participant_health_status'].lower() or
                                                                   'получил травмы' in x['participant_health_status'].lower() or
                                                                   'телесные повреждения' in x['participant_health_status'].lower()
                                                         else 0, 
                                                axis=1)

Проверим полученный результат:

In [170]:
participant_df.head()

,id,vehicle_year,vehicle_brand,vehicle_color,vehicle_model,vehicle_category,participant_role,participant_gender,participant_violations,participant_health_status,participant_years_of_driving_experience,is_intoxication,is_use_mobcell,is_above_speed_limit,is_traffic_light_violation,is_dead,is_not_injured,is_injured
0,1504730,2012,KIA,Серый,Venga,"В-класс (малый) до 3,9 м",Водитель,Женский,Выезд на полосу встречного движения,Не пострадал,4.0,0,0,0,0,0,1,0
1,1504730,2012,KIA,Серый,Venga,"В-класс (малый) до 3,9 м",Пассажир,Женский,,Не пострадал,NaN,0,0,0,0,0,1,0
2,1504730,2013,Прочие марки мотоциклов,Красный,Прочие марки мотоциклов,Мопеды с двигателем внутреннего сгорания менее 50 см. куб.,Водитель,Мужской,"Управление ТС лицом, не имеющим права на управление ТС",Скончался на месте ДТП до приезда скорой медицинской помощи,NaN,0,0,0,0,1,0,0
3,156565,2011,KIA,Синий,Rio,"D-класс (средний) до 4,6 м",Водитель,Женский,Другие нарушения ПДД водителем,"Раненый, находящийся (находившийся) на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)",10.0,0,0,0,0,0,0,1
4,156274,2009,ВАЗ,Черный,Priora,"В-класс (малый) до 3,9 м",Водитель,Мужской,"Оставление места ДТП, Несоответствие скорости конкретным условиям движения",Не пострадал,13.0,0,0,0,0,0,1,0


## 3. Исследовательский анализ данных

## 4. Проверка гипотез

## 5. Общие выводы по исследованию